In [1]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_recall_curve, accuracy_score
from sklearn.metrics import class_likelihood_ratios, classification_report

import plotly.express as px
import pandas as pd
import numpy as np

from dataclasses import dataclass, field
from typing import List
import json

## Data loading and organization

We'll grab in the data file and pull it in, isolate relevant columns, and do a quick sanity check of visualization to ensure that we grabbed the right data.

In [2]:
df = pd.read_parquet('../../data/converted/harmonized_allwaves.parquet')
print(len(df))

7533


In [3]:
c_cols = [c for c in df.columns if c.startswith('cl') and "_" not in c]

y_cols = ["dcany", "dcanyanx", "dcanydep", "dcanyhk", "dcanycd", "dcsepa",
          "dcspph", "dcsoph", "dcpanic", "dcagor", "dcptsd", "dcocd",
          "dcgena", "dcdmdd", "dcmadep", "dcmania", "dcodd", "dccd"]

# N.B.: Hard limit with current training procedure is N=25 with a given diagnostic label
#       As a result, we have to drop: dcagor, dcdmdd, dcmania.
#       We will also drop the following, which have less than 150, due to too small a sample, regardless:
#            dcocd, dcpanic, dcsepa, dcspph, dcsoph, dcptsd, dcgena, dcccd, dcmadep

y_cols = ["dcany", "dcanyanx", "dcanydep", "dcanyhk", "dcanycd", "dcodd", "dcsepa", "dcmadep", "dcspph"]

x_cols = list(set(c_cols) - set(y_cols))

In [4]:
df = df[x_cols + y_cols].dropna(axis=0, how='any')
df[y_cols] = df[y_cols].replace({2.0:1, 0.0:0})

In [5]:
tmp = []
for yc in y_cols:
    vc = df[yc].value_counts()
    tmp += [{"Dx": yc, "HC": vc.loc[0.0], "Pt":vc.loc[1.0]}]
pd.DataFrame.from_dict(tmp)

,Dx,HC,Pt
0,dcany,3757,1228
1,dcanyanx,4378,607
2,dcanydep,4687,298
3,dcanyhk,4611,374
4,dcanycd,4701,284
5,dcodd,4808,177


## Learning in a 'Complete' data setting

As a sanity check and general "test" of the training and testing approach, we are training a learner with all of the CBCL items to repeatedly classify participants on one of the diagnostic categories using:

1. A stratified outer cross-validation loop (5-fold)
2. An calibrated inner cross-validation loop with voting procedure (5-fold; 25 folds combined)

In this case, we are using all of our features/survey questions, so this will set the ceiling for our performance. We expect that this out-of-sample performance is nearly quite high given the strong relationship beetween survey response data and diagnostic scores.

Importantly, we will use the feature importance scores from these classifiers as the staring point for us reducing our feature set in the next stage.

In [6]:
@dataclass
class Learner:
    """Data class to record and present statistics from trained & evaluated models"""
    dx: str  # Diagnosis
    hc_n: int  # Number of healthy controls
    dx_n: int  # Number of patients
    x_ids: List = field(default_factory=lambda: [])  # List of features used in the learner
    fi: List = field(default_factory=lambda: [])  # Feature importance lists
    f1: List = field(default_factory=lambda: [])  # F1 score lists
    sen: List = field(default_factory=lambda: [])  # Sensitivity score lists
    spe: List = field(default_factory=lambda: [])  # Specificity score lists
    LRp: List = field(default_factory=lambda: [])  # Positive likelihood ratio lists
    LRn: List = field(default_factory=lambda: [])  # Negative likelihood ratio lists
    acc_train: List = field(default_factory=lambda: [])  # Performance on the training set
    acc_valid: List = field(default_factory=lambda: [])  # Performance on the validation set

    proba: List = field(default_factory=lambda: [])  # Prediction probability/confidence
    label: List = field(default_factory=lambda: [])  # Prediction labels

    def summary(self, show=False):
        """Prints a summary report"""
        self._sanitize()
        
        tmpdict = [{
            'DX': self.dx,
            'N_HC': self.hc_n,
            'N_Pt': self.dx_n,
            'N_xs': len(self.x_ids),
            'F1': np.mean(self.f1),
            'Sensitivity': np.mean(self.sen),
            'Specificity': np.mean(self.spe),
            'LR+': np.mean(self.LRp),
            'LR-': np.mean(self.LRn),
            'Accuracy (Train)': np.mean(self.acc_train),
            'Accuracy (Validation)': np.mean(self.acc_valid)
            
        }]
        tmpdf = pd.DataFrame.from_dict(tmpdict)

        if show:
            self._plot_probas()
            print(tmpdf)

        return tmpdf
    
    def _sanitize(self):
        """Make lists of lists a bit more palletable..."""
        self.proba = np.vstack(self.proba)
        self.label = np.hstack(self.label)
    
    def _plot_probas(self):
        """Show prediction confidence scores"""
        tmpdict = {
            'proba' : self.proba[:,1],            
            'label' : self.label
        }
        tmpdf = pd.DataFrame.from_dict(tmpdict)
        fig = px.histogram(tmpdf, color='label', x='proba', marginal='box',
                           barmode="overlay", opacity=0.7, category_orders={'label':[0,1]})
        fig.show()

In [7]:
def fit_models(df, x_ids, y_ids, verbose=True):
    """General purpose function for fitting callibrated classifiers for Dx from Survey data"""
 
    # Establish Models & Cross-Validation Strategy
    #   Base classifier: Random Forest. Rationale: non-parametric, has feature importance
    clf_rf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
    #   CV: Stratified K-fold. Rationale: shuffle data, balance classes across folds
    cv = StratifiedKFold(shuffle=True, random_state=42)
    #   Top-level classifier: Calibrated CV classifier. Rationale: prioritizes maintaining class balances
    clf_calib = CalibratedClassifierCV(estimator=clf_rf, cv=cv)
    np.random.seed(42)

    # Create X (feature) matrix: grab relevant survey columns x rows from dataframe
    X = df[x_ids].values

    # Create empty list of learners
    learners = []
    summaries = []
    # For every Dx that we want to predict...
    for y_name in y_ids:
        # Create the y (target) matrix/vector: grab relevant Dx rows from dataframe
        y = df[y_name].values.astype(int)
        
        # Get Pt and HC counts from dataframe, and initialize the learner object
        _, uc = np.unique(y, return_counts=True)

        if verbose:
            print(f"Dx: {y_name} | HC: {uc[0]} | Pt: {uc[1]}")

        current_learner = Learner(dx=y_name, hc_n=uc[0], dx_n=uc[1], x_ids=x_ids)
        # Set-up a CV loop (note, we use the same CV strategy both within the Calibrated CLF and here,
        #   resulting in nested-stratified-k-fold-CV)
        for idx_train, idx_test in cv.split(X, y):
            # Split the dataset into train and test sets
            X_tr = X[idx_train, :]
            y_tr = y[idx_train]

            X_te = X[idx_test, :]
            y_te = y[idx_test]

            # Fit the callibrated classifier on the training data
            clf_calib.fit(X_tr, y_tr)

            # Extract/Generate relevant data from (all internal folds of) the classifier...
            # Make predictions on the test set
            y_pred = clf_calib.predict(X_te)
            
            # Grab training and validation performance using the integrated scoring function (accuracy)
            y_pred_tr = clf_calib.predict(X_tr)
            current_learner.acc_train.append(accuracy_score(y_tr, y_pred_tr))
            current_learner.acc_valid.append(accuracy_score(y_te, y_pred))

            # Grab feature importance scores
            fis = [_.estimator.feature_importances_ for _ in clf_calib.calibrated_classifiers_]
            current_learner.fi.append(fis)

            # Grab the prediction probabilities
            current_learner.proba.append(clf_calib.predict_proba(X_te))
            current_learner.label.append(y_te)

            # Grab the prediction labels
            current_learner.f1.append(f1_score(y_te, y_pred))

            # Grab the sensitivity and specificity (i.e. recall of each of Dx and HC classes)
            report_dict = classification_report(y_te, y_pred, output_dict=True)
            current_learner.sen.append(report_dict['1']['recall'])
            current_learner.spe.append(report_dict['0']['recall'])

            # Grab the positive/negative likelihood ratios
            lrp, lrn = class_likelihood_ratios(y_te, y_pred)
            current_learner.LRp.append(lrp)
            current_learner.LRn.append(lrn)

        # Summarize current learner performance, save it, and get ready to go again!
        summaries += [current_learner.summary()]
        learners += [current_learner]
        del current_learner
    
    summaries = pd.concat(summaries)
    return learners, summaries

In [8]:
x_ids = x_cols
y_ids = y_cols
df = df

learners, summaries = fit_models(df, x_ids, y_ids)
summaries

Dx: dcany | HC: 3761 | Pt: 1231
Dx: dcanyanx | HC: 4383 | Pt: 609
Dx: dcanydep | HC: 4692 | Pt: 300
Dx: dcanyhk | HC: 4618 | Pt: 374
Dx: dcanycd | HC: 4709 | Pt: 283
Dx: dcodd | HC: 4815 | Pt: 177


,DX,N_HC,N_Pt,N_xs,F1,Sensitivity,Specificity,LR+,LR-,Accuracy (Train),Accuracy (Validation)
0,dcany,3761,1231,121,0.547586,0.447605,0.938846,7.465590,0.588521,0.996695,0.817708
0,dcanyanx,4383,609,121,0.134793,0.077130,0.991558,9.449985,0.930705,0.997947,0.880008
0,dcanydep,4692,300,121,0.172807,0.103333,0.995098,20.957556,0.901038,0.999449,0.941506
0,dcanyhk,4618,374,121,0.355576,0.254162,0.986356,20.086077,0.756090,0.999599,0.931491
0,dcanycd,4709,283,121,0.429023,0.317920,0.990442,42.546086,0.688699,0.999800,0.952322
0,dcodd,4815,177,121,0.116822,0.067619,0.996885,27.234048,0.935296,0.999800,0.963942


## Visualize feature importance scores

Let's look at which features are most predictive for each of the clinical targets. We take the average feature importance scores across the 25 trained models, and stack them. We can then begin to identify which features are most important.

In [92]:
df_learners = []
NQ = 20

for l in learners:
    tmp_dict = {
        "dx": l.dx
    }

    means = np.mean(np.vstack(l.fi), axis=0)
    for assessment, importance in zip(x_cols, means):
        tmp_dict[assessment] = importance

    df_learners += [tmp_dict]

df_learners = pd.DataFrame.from_dict(df_learners)
df_learners_melt = df_learners.melt(id_vars=['dx'], value_vars=x_cols, value_name='importance')

In [93]:
fig = px.bar(df_learners_melt, x="variable", y="importance", color="dx",
             title="Relative feature importance of survey questions across diagnoses",
             labels={"variable": "CBCL Question",
                     "importance": "Relative Feature Importance",
                     "target": "Diagnosis"},
             template='plotly_white')
sort1 = (df_learners_melt
         .groupby('variable')
         .sum()
         .reset_index()
         .sort_values('importance')['variable'].values[::-1])

fig.update_xaxes(categoryorder='array', categoryarray=sort1)
fig.update_layout(width=1620, height=900, legend={'orientation': 'v','y':0.97,'x':0.84})
fig.add_vline(x=sort1[NQ+1], line_width=1, line_dash="dash", line_color="gray")
fig.show()
fig.write_image('../../data/feature_importance.png')

/home/gkiar/env/gp311/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [94]:
fi_qs = sort1[0:NQ]
print(f"The {NQ} most predictive survey questions overall are:", ", ".join(fi_qs))

The 20 most predictive survey questions overall are: cl8, cl10, cl86, cl95, cl4, cl28, cl22, cl87, cl78, cl41, cl3, cl88, cl45, cl103, cl43, cl61, cl50, cl23, cl5, cl35


## Consistency of important features

Next, we want to look at the consistency with which each of the most impactful features is consistently impactful across each of the diagnostic categories. To do this, we're calculating a *Top-N Fraction* measure that captures the fraction of the time a given question lies within the top *N* questions with respect to their feature importance across all diagnoses.

For example, if a question is the most predictive (Top-1) for a single diagnosis but none of the others, it would receive a score of 1 / total number of diagnoses (in this case, 1/13). If a question were always the most important, it would receive a score of 1. Looking further down the table, at Top-10, each question that is within the ten most predictive questions for a given diagnosis will have a score of at least 1 / total number of diagnoses, and will similarly increase based on the number of Top-1o lists it appears on, and so-forth.

In [95]:
topN = len(x_cols)
item_relevance = np.empty((topN, len(x_cols)))
for n in range(topN):
    item_relevance[n, :] = (df_learners[x_cols]
                            .rank(axis=1, ascending=False)
                            .apply(lambda x: x <= n+1)
                            .sum(axis=0))

In [96]:
idx = np.lexsort(item_relevance[::-1,:])[::-1]
sort2 = np.array(x_cols)[idx]

fig = px.imshow(item_relevance[:, idx]*1.0/len(y_cols),
                x=sort2, y=np.arange(topN)+1,
                labels={'x':'CBCL Question',
                        'y':'Number of Questions (N)',
                        'color':'Top-N Fraction'},
                title="Consistency of question usefulness across diagnoses",
                template='plotly_white')
fig.update_layout(width=1620, height=900)
fig.add_vline(x=sort2[NQ+1], line_width=1, line_dash="dash", line_color="gray")
fig.write_image('../../data/topn_importance.png')
fig.show()

In [97]:
topn_qs = sort2[0:NQ]
print(f"The {NQ} most commonly useful survey questions overall are:", ", ".join(topn_qs))

The 20 most commonly useful survey questions overall are: cl8, cl28, cl86, cl103, cl50, cl4, cl22, cl95, cl10, cl54, cl9, cl41, cl78, cl35, cl47, cl45, cl5, cl112, cl87, cl14


In [98]:
all_qs = list(set(list(topn_qs) + list(fi_qs)))
diff = np.abs(NQ-len(all_qs))
frac = 1.0*diff/NQ*100
print(f"The two lists differ by {diff} / {NQ} items ({frac:.2f}%)")

The two lists differ by 5 / 20 items (25.00%)


In [100]:
qs = {"fi": list(fi_qs), "topN": list(topn_qs)}
with open('../../data/qs_20.json', 'w') as fhandle:
    json.dump(qs, fhandle)

## Making predictions with fewer instruments

Finally, the reason we're here: to see if, by using the most impactful questions, we can produce predictions of similar quality to the complete battery of questions. We sorted the question list based on an average position from the two sorting strategies we attempted, and will increasingly drop questions from the instrument and evaluate its performance for each diagnosis.

In [20]:
avg_rank = np.mean(np.where(sort1[:, None] == sort2), axis=0)
avg_sorted = sort1[np.argsort(avg_rank)]

In [111]:
with open('../../data/qs_20.txt', 'w') as fhandle:
    fhandle.writelines("\n".join(avg_sorted[0:20]))

In [ ]:
x_ids = avg_sorted
y_ids = y_cols
df = df

degrading_learners = []
degrading_summaries = []
print("Number of questions used: ", end="")
for n_questions in range(len(x_ids))[::-1]:
    xi = x_ids[0:n_questions+1]
    X = df[xi].values
    
    print(",", n_questions+1, end=" ")
    dl, sm = fit_models(df, xi, y_ids, verbose=False)
    degrading_learners += [dl]
    degrading_summaries += [sm]
print(".")

degrading_summaries = pd.concat(degrading_summaries)
degrading_summaries

In [ ]:
degrading_summaries.to_parquet("../../data/subsample_performance.parquet")
# TODO: write out degrading_learners

In [31]:
degrading_summaries = pd.read_parquet('../../data/subsample_performance.parquet')

In [33]:
rename_dict = {'N_xs': "Number of CBCL Questions",
               'DX':'Dx',
               'variable': 'Measure',
               'value': 'Performance'}

melted_summaries = pd.melt(degrading_summaries, id_vars=['DX', 'N_HC', 'N_Pt', 'N_xs'])
melted_summaries.rename(columns=rename_dict, inplace=True)
melted_summaries

,Dx,N_HC,N_Pt,Number of CBCL Questions,Measure,Performance
0,dcany,3761,1231,121,F1,0.546442
1,dcanyanx,4383,609,121,F1,0.124020
2,dcanydep,4692,300,121,F1,0.208427
3,dcanyhk,4618,374,121,F1,0.346435
4,dcanycd,4709,283,121,F1,0.418993
...,...,...,...,...,...,...
5077,dcanyanx,4383,609,1,Accuracy (Validation),0.878005
5078,dcanydep,4692,300,1,Accuracy (Validation),0.939904
5079,dcanyhk,4618,374,1,Accuracy (Validation),0.925080
5080,dcanycd,4709,283,1,Accuracy (Validation),0.943309


In [101]:
fig = px.line(melted_summaries,
              color='Dx',
              x='Number of CBCL Questions',
              y='Performance',
              facet_col='Measure',
              facet_col_wrap=2,
              category_orders={'Measure':['LR+', 'LR-',
                                          'Sensitivity', 'Specificity',
                                          'Accuracy (Train)', 'Accuracy (Validation)',
                                          'F1']},
              width=1200,
              height=1600,
              template='plotly_white')

fig.update_yaxes(matches=None)
fig.update_layout(xaxis={'autorange':'reversed'})
fig.show()
fig.write_image('../../data/degrading_performance.png')

In [62]:
degrading_summaries.query('N_xs == 121')

,DX,N_HC,N_Pt,N_xs,F1,Sensitivity,Specificity,LR+,LR-,Accuracy (Train),Accuracy (Validation)
0,dcany,3761,1231,121,0.546442,0.446786,0.938313,7.299504,0.589655,0.996695,0.817107
0,dcanyanx,4383,609,121,0.124020,0.070560,0.991786,8.905762,0.937103,0.997947,0.879407
0,dcanydep,4692,300,121,0.208427,0.126667,0.995311,30.012540,0.877422,0.999449,0.943109
0,dcanyhk,4618,374,121,0.346435,0.246162,0.986139,19.533749,0.764422,0.999599,0.930688
0,dcanycd,4709,283,121,0.418993,0.314474,0.988744,39.435129,0.693340,0.999800,0.950520
0,dcodd,4815,177,121,0.075368,0.044603,0.995846,15.285714,0.959412,0.999800,0.962139


In [71]:
diagnosis = degrading_summaries['DX'].unique()
diagnosis

array(['dcany', 'dcanyanx', 'dcanydep', 'dcanyhk', 'dcanycd', 'dcodd'],
      dtype=object)

In [79]:
diagnosis = degrading_summaries['DX'].unique()
measure = melted_summaries['Measure'].unique()

dflist = []
for d in diagnosis:
    tmpdf = degrading_summaries.query(f'DX == "{d}"')
    for m in measure:
        tmpdf[m] = tmpdf[m] / tmpdf[m].iloc[0]
    dflist += [tmpdf]

normalized_summaries = pd.concat(dflist)
normalized_summaries

/tmp/ipykernel_3959/1568193032.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DX,N_HC,N_Pt,N_xs,F1,Sensitivity,Specificity,LR+,LR-,Accuracy (Train),Accuracy (Validation)
0,dcany,3761,1231,121,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
0,dcany,3761,1231,120,0.998064,1.003639,0.997451,0.967268,0.999658,1.000000,0.998285
0,dcany,3761,1231,119,0.996132,0.998195,0.998583,0.979566,1.002856,1.000000,0.998529
0,dcany,3761,1231,118,0.998660,1.003661,0.997734,0.966770,0.999235,1.000000,0.998530
0,dcany,3761,1231,117,0.992308,0.989089,1.000284,0.991422,1.008543,1.000000,0.998775
...,...,...,...,...,...,...,...,...,...,...,...
0,dcodd,4815,177,5,0.000000,0.000000,1.004171,NaN,1.042305,0.964737,1.002499
0,dcodd,4815,177,4,0.000000,0.000000,1.004171,NaN,1.042305,0.964737,1.002499
0,dcodd,4815,177,3,0.000000,0.000000,1.004171,NaN,1.042305,0.964737,1.002499
0,dcodd,4815,177,2,0.000000,0.000000,1.004171,NaN,1.042305,0.964737,1.002499


In [ ]:
normalized_summaries

In [73]:
rename_dict = {'N_xs': "Number of CBCL Questions",
               'DX':'Dx',
               'variable': 'Measure',
               'value': 'Performance'}

melted_normalized_summaries = pd.melt(normalized_summaries, id_vars=['DX', 'N_HC', 'N_Pt', 'N_xs'])
melted_normalized_summaries.rename(columns=rename_dict, inplace=True)
melted_normalized_summaries

,Dx,N_HC,N_Pt,Number of CBCL Questions,Measure,Performance
0,dcany,3761,1231,121,F1,1.000000
1,dcany,3761,1231,120,F1,0.998064
2,dcany,3761,1231,119,F1,0.996132
3,dcany,3761,1231,118,F1,0.998660
4,dcany,3761,1231,117,F1,0.992308
...,...,...,...,...,...,...
5077,dcodd,4815,177,5,Accuracy (Validation),1.002499
5078,dcodd,4815,177,4,Accuracy (Validation),1.002499
5079,dcodd,4815,177,3,Accuracy (Validation),1.002499
5080,dcodd,4815,177,2,Accuracy (Validation),1.002499


In [106]:
tmpdf = melted_normalized_summaries.groupby(['Dx', 'Number of CBCL Questions'])['Performance'].median().reset_index()
tmpdf.rename(columns={'Performance': 'Relative Performance'}, inplace=True)

fig = px.line(tmpdf, x='Number of CBCL Questions',y='Relative Performance', color='Dx',
              width=1620, height=900, template='plotly_white')

fig.update_layout(xaxis={'autorange':'reversed'})
fig.add_vline(x=NQ, line_width=1, line_dash="dash", line_color="gray")

fig.show()
fig.write_image('../../data/relative_performance.png')


/home/gkiar/env/gp311/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [105]:
fig = px.line(melted_normalized_summaries,
              color='Dx',
              x='Number of CBCL Questions',
              y='Performance',
              facet_col='Measure',
              facet_col_wrap=2,
              category_orders={'Measure':['LR+', 'LR-',
                                          'Sensitivity', 'Specificity',
                                          'Accuracy (Train)', 'Accuracy (Validation)',
                                          'F1']},
              width=1200,
              height=1600,
              template='plotly_white')

# fig.update_yaxes(matches=None)
fig.update_layout(xaxis={'autorange':'reversed'})
fig.show()
fig.write_image('../../data/degrading_performance_normalized.png')

In [69]:
px.line(tmpdf, x='N_xs', y='Norm LR+')

## Total number of models

* Calibrated Classifier: 5x
* Nested CV: 5x
* Dx categories: 6x
* Survey size: 121x

* Total = 18,150

In [ ]:
from plotly.subplots import make_subplots
self = current_learner
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3],
                    subplot_titles=("First Subplot","Second Subplot"))

tmpdict = {
    'proba' : self.proba[:,1],
    'label' : self.label
}
tmpdf = pd.DataFrame.from_dict(tmpdict)
tf = px.box(tmpdf, color='label', x='proba', notched=True, points='outliers',category_orders={'label':[0,1]})

fig.add_traces(tf.data, cols=1, rows=1)
# fig.add_trace(tf.data[1], col=1, row=1)

p, r, t = precision_recall_curve(self.label, self.proba[:,1], pos_label=1)
tmpdict = {
    '1-Recall' : 1-r,
    'Precision' : p,
}
tmpdf = pd.DataFrame.from_dict(tmpdict)
tf = px.line(tmpdf, x='1-Recall', y='Precision')

fig.add_trace(tf.data[0], col=2, row=1)
fig.show()
